<a href="https://colab.research.google.com/github/beifa/Bengali.AI/blob/master/luke_i_am_your_father.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Private score 0.0386 ))))

Это мой первый шаг.

In [0]:
import numpy as np
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import gc
import torch
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F

/kaggle/input/bengaliai-cv19/test_image_data_0.parquet
/kaggle/input/bengaliai-cv19/sample_submission.csv
/kaggle/input/bengaliai-cv19/test_image_data_3.parquet
/kaggle/input/bengaliai-cv19/train_image_data_0.parquet
/kaggle/input/bengaliai-cv19/test_image_data_1.parquet
/kaggle/input/bengaliai-cv19/train_image_data_2.parquet
/kaggle/input/bengaliai-cv19/train_image_data_3.parquet
/kaggle/input/bengaliai-cv19/test_image_data_2.parquet
/kaggle/input/bengaliai-cv19/train_image_data_1.parquet
/kaggle/input/bengaliai-cv19/class_map.csv
/kaggle/input/bengaliai-cv19/test.csv
/kaggle/input/bengaliai-cv19/train.csv
/kaggle/input/bengali64/test_data_1.feather
/kaggle/input/bengali64/test_data_2.feather
/kaggle/input/bengali64/test_data_0.feather
/kaggle/input/bengali64/train_data_0.feather
/kaggle/input/bengali64/train_data_1.feather
/kaggle/input/bengali64/train_data_2.feather
/kaggle/input/bengali64/train_data_3.feather
/kaggle/input/bengali64/test_data_3.feather


In [0]:
sample_submission = pd.read_csv("../input/bengaliai-cv19/sample_submission.csv")
test = pd.read_csv("../input/bengaliai-cv19/test.csv")
train = pd.read_csv("../input/bengaliai-cv19/train.csv")

In [0]:
images = pd.read_feather('/kaggle/input/bengali64/train_data_0.feather')

In [0]:
# train[:50210]

In [0]:
#137x236 grayscale images
# images.iloc[0, 1:].values.reshape(137, 236).shape

In [0]:
class Dataset(Dataset):
    def __init__(self, data, label):
        self.data = data
        self.label = label
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        #label this is train
        lab1 = self.label.grapheme_root.values[idx]
        lab2 = self.label.vowel_diacritic.values[idx]
        lab3 = self.label.consonant_diacritic.values[idx]
        img = self.data.iloc[idx, 1:].values.reshape(64, 64).astype('float')
        return img, lab1, lab2, lab3

In [0]:
# data = Dataset(images, train[:50210])
# dataloader = DataLoader(data, batch_size=64,
#                         shuffle=True)

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 32, 3)
        #self.banch1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(32, 16, 3)
        # an affine operation: y = Wx + b
        #self.banch2 = nn.BatchNorm2d(32)
        self.avgpool = nn.AvgPool2d(2)
        self.fc0 = nn.Linear(784, 200)  # 6*6 from image dimension
        self.fc1 = nn.Linear(200, 168)
        self.fc2 = nn.Linear(200, 11)
        self.fc3 = nn.Linear(200, 7)
        

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        #x = self.banch1(x)
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        #x = self.banch2(x)
        x = self.avgpool(x)                        
        x = x.view(-1, self.num_flat_features(x))
        x = self.fc0(x)        
        x1 = self.fc1(x)
        x2 = self.fc2(x)
        x3 = self.fc3(x)
        return x1, x2, x3
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1))
  (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc0): Linear(in_features=784, out_features=200, bias=True)
  (fc1): Linear(in_features=200, out_features=168, bias=True)
  (fc2): Linear(in_features=200, out_features=11, bias=True)
  (fc3): Linear(in_features=200, out_features=7, bias=True)
)


In [0]:
16*6*6

576

In [0]:
optimizer = torch.optim.Adam(net.parameters(), lr=0.1)
criterion = nn.CrossEntropyLoss()

In [0]:
from tqdm import tqdm
running_loss = 0
running_acc = 0.0

reduced_index = train[:50210].groupby(['grapheme_root', 'vowel_diacritic', 'consonant_diacritic']).apply(lambda x: x.sample(5)).image_id.values
reduced_train = train.loc[train.image_id.isin(reduced_index)]
reduced_data = images.loc[images.image_id.isin(reduced_index)]

data = Dataset(reduced_data, reduced_train)
dataloader = DataLoader(data, batch_size=32, shuffle=True)


for idx, (img,lab1,lab2,lab3) in tqdm(enumerate(dataloader),total=len(dataloader)):
    net.train()
    optimizer.zero_grad()
    outputs1,outputs2,outputs3 = net(img.unsqueeze(1).float())
    loss1 = criterion(outputs1,lab1)
    loss2 = criterion(outputs2,lab2)
    loss3 = criterion(outputs3,lab3)
    running_loss += loss1+loss2+loss3
    
    running_acc += (outputs1.argmax(1)==lab1).float().mean()
    running_acc += (outputs2.argmax(1)==lab2).float().mean()
    running_acc += (outputs3.argmax(1)==lab3).float().mean()
    (loss1+loss2+loss3).backward()
    optimizer.step()
    
print(running_loss/len(dataloader))
print(running_acc/(len(dataloader)*3))  

100%|██████████| 202/202 [01:05<00:00,  3.07it/s]

tensor(201.8296, grad_fn=<DivBackward0>)
tensor(0.2716)


In [0]:
class GraphemeDataset(Dataset):
    def __init__(self,df,_type='train'):
        self.df = df
    def __len__(self):
        return len(self.df)
    def __getitem__(self,idx):
        image = self.df.iloc[idx][1:].values.reshape(64,64).astype(float)
        return image

In [0]:
import cv2
def Resize(df,size=64):
    resized = {} 
    df = df.set_index('image_id')
    for i in tqdm(range(df.shape[0])):
        image = cv2.resize(df.loc[df.index[i]].values.reshape(137,236),(size,size))
        resized[df.index[i]] = image.reshape(-1)
    resized = pd.DataFrame(resized).T.reset_index()
    resized.columns = resized.columns.astype(str)
    resized.rename(columns={'index':'image_id'},inplace=True)
    return resized

In [0]:
net.eval()
test_data = ['test_image_data_0.parquet','test_image_data_1.parquet','test_image_data_2.parquet','test_image_data_3.parquet']
predictions = []
batch_size=1
for fname in test_data:
    data = pd.read_parquet(f'/kaggle/input/bengaliai-cv19/{fname}')
    data = Resize(data)
    test_image = GraphemeDataset(data)
    test_loader = torch.utils.data.DataLoader(test_image,batch_size=1,shuffle=False)
    with torch.no_grad():
        for idx, (inputs) in tqdm(enumerate(test_loader),total=len(test_loader)):      
            
            outputs1,outputs2,outputs3 = net(inputs.unsqueeze(1).float())
            predictions.append(outputs3.argmax(1).cpu().detach().numpy())
            predictions.append(outputs2.argmax(1).cpu().detach().numpy())
            predictions.append(outputs1.argmax(1).cpu().detach().numpy())

100%|██████████| 3/3 [00:00<00:00, 107.57it/s]


In [0]:
submission = pd.read_csv('/kaggle/input/bengaliai-cv19/sample_submission.csv')

In [0]:
submission.target = np.hstack(predictions)
submission.head(10)

,row_id,target
0,Test_0_consonant_diacritic,0
1,Test_0_grapheme_root,0
2,Test_0_vowel_diacritic,133
3,Test_1_consonant_diacritic,0
4,Test_1_grapheme_root,0
5,Test_1_vowel_diacritic,133
6,Test_2_consonant_diacritic,0
7,Test_2_grapheme_root,0
8,Test_2_vowel_diacritic,133
9,Test_3_consonant_diacritic,0


In [0]:
submission.to_csv('submission.csv',index=False)